## Workflow to update a TX county data layer with the latest COVID-19 cases
Still a work in progress...

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import geopandas as gpd
import pandas as pd
import numpy as np
from arcgis.gis import GIS
from arcgis.features import FeatureLayer

In [ ]:
# scrape Texas Department of State Health Services
ssl._create_default_https_context = ssl._create_unverified_context # avoid cert error 
url = 'https://www.dshs.state.tx.us/news/updates.shtm#coronavirus'
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# get count of latest COVID-19
list_for_df = []
for tr in soup.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    list_for_df.append(row)

# create df with counts, use filter() to remove empty list
count_df = pd.DataFrame(list(filter(None,list_for_df)), columns=['CNTY_NM', 'COUNT'])

In [ ]:
# read in shapefile using GeoPandas
county_gdf_raw = gpd.GeoDataFrame(gpd.read_file(r'data/origs/Texas_County_Boundaries.shp'))
county_gdf = county_gdf_raw[['OBJECTID','CNTY_NM','geometry']]
county_gdf.head()

In [ ]:
# merge count_df and county_gdf to add COVID19 count to geometry
merged_df = county_gdf.merge(count_df, left_on='CNTY_NM', right_on='CNTY_NM', how='left')
# convert "COUNT" field to int
merged_df["COUNT"] = merged_df.COUNT.fillna(0)
merged_df["COUNT"] = merged_df.COUNT.astype(np.int64)

In [ ]:
# make sure projection looks good
%matplotlib inline
merged_df.plot()

In [ ]:
# write to file
# dosen't create a .prj file, so used the one from the original data
merged_df.to_file('data/counties_count.shp')

In [ ]:
## values for GIS()
f = open('agol.csv')
username = str(f.readline())[:-1]
password = str(f.readline())[:-1]

In [ ]:
# add to AGOL 
gis = GIS("https://www.arcgis.com", username, password)

tx_county_covid19_properties = {
    'title': 'TX Counties COVID19 Count',
    'tags': 'tx, open data',
    'type': 'Shapefile'
}

In [ ]:
# load the shapefile
data_file_location = r'data/tx_counties_covid19.zip'
tx_county_covid19_shp = gis.content.add(tx_county_covid19_properties, data=data_file_location)

In [ ]:
# call publish method, which returns another arcgis.gis.Item instance for the feature layer
tx_county_covid19_feature_layer_item = tx_county_covid19_shp.publish()

tx_county_covid19_feature_layer_item


### The above workflow got the data on AGOL
### Below, the created feature layer will be updated daily

In [ ]:
# access the layer to edit it
published_covid19 = gis.content.search('title:TX Counties COVID19 Count')
published_covid19_lyr = published_covid19[0].layers


In [ ]:
# create a featureset to access the data
featureSet_published_covid19_lyr = published_covid19_lyr[0].query()

In [ ]:
# create a pandas df to make updating eaiser
fset_df = featureSet_published_covid19_lyr.sdf
fset_df.columns

In [ ]:
# next will need to edit the above df and then publish again
# 
published_covid19_lyr[0].properties.capabilities